In [ ]:
import xarray as xr
import numpy as np
import logging
from pathlib import Path
import geopandas as gpd
from pydantic import ConfigDict
from dotenv import dotenv_values

from damo_afvoergebiedaanvoergebied import GeneratorBasis

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
class GeneratorAlles(GeneratorBasis):
    model_config = ConfigDict(arbitrary_types_allowed=True)

    path: Path = None
    name: str = None
    base_dir: Path = None
    waterschap: str = None

    hydroobject: gpd.GeoDataFrame = None
    overige_watergang: gpd.GeoDataFrame = None

    bebouwing: gpd.GeoDataFrame = None
    kering: gpd.GeoDataFrame = None
    nwb: gpd.GeoDataFrame = None
    peilgebied: gpd.GeoDataFrame = None
    snelweg: gpd.GeoDataFrame = None
    spoorweg: gpd.GeoDataFrame = None

    hydroobject: gpd.GeoDataFrame = None
    overige_watergang: gpd.GeoDataFrame = None
    splitsing: gpd.GeoDataFrame = None
    outflow_nodes: gpd.GeoDataFrame = None
    specifieke_afvoer: xr.Dataset = None

    required_results: list[str] = [
        "hydroobject_processed_0", 
        "hydroobject_processed_1", 
        "overige_watergang_processed_3", 
        "overige_watergang_processed_4", 
        "outflow_nodes_overige_watergang",
        "potential_culverts_5", 
        "ghg_processed",
        "afvoergebied_0",
        "afvoergebied_1",
        "afvoergebied_2",
        "afvoergebied_3",
        "afvoergebied_4",
        "nodes",
        "edges",
    ]
    
    hydroobject_processed_0: gpd.GeoDataFrame = None
    hydroobject_processed_1: gpd.GeoDataFrame = None

    overige_watergang_processed_3: gpd.GeoDataFrame = None
    overige_watergang_processed_4: gpd.GeoDataFrame = None
    outflow_nodes_overige_watergang: gpd.GeoDataFrame = None
    potential_culverts_5: gpd.GeoDataFrame = None

    ghg_processed: xr.Dataset = None

    afvoergebied_0: xr.Dataset = None
    afvoergebied_0_gdf: gpd.GeoDataFrame = None
    afvoergebied_1: xr.Dataset = None
    afvoergebied_1_gdf: gpd.GeoDataFrame = None
    afvoergebied_2: xr.Dataset = None
    afvoergebied_2_gdf: gpd.GeoDataFrame = None
    afvoergebied_3: xr.Dataset = None
    afvoergebied_3_gdf: gpd.GeoDataFrame = None
    afvoergebied_4: xr.Dataset = None
    afvoergebied_4_gdf: gpd.GeoDataFrame = None

    split_nodes: gpd.GeoDataFrame = None

    edges: gpd.GeoDataFrame = None
    nodes: gpd.GeoDataFrame = None

In [ ]:
# Define case and base_dir
config = dotenv_values("..\\.env")

base_dir = Path(config["PATH_BASIS"])
waterschap = config["WATERSCHAP"]
case_name = config["CASE_NAME"]
dir_basisdata = config["DIR_NAME_BASISDATA"]
dir_results = config["DIR_NAME_RESULTATEN"]

case_path = Path(base_dir, case_name)

In [ ]:
res = GeneratorAlles(
    path=case_path, 
    dir_basisdata=dir_basisdata,
    dir_results=dir_results,
    waterschap=waterschap,
    read_results=True, 
    write_results=True,
)

In [ ]:
edges = res.edges.copy()
edges.geometry = edges.geometry.interpolate(0.5, normalized=True)

In [ ]:
edges["nieuwe_categorie"] = 3
edges.loc[edges["total_specifieke_afvoer"]>=0.010, "nieuwe_categorie"] = 2
edges.loc[edges["total_specifieke_afvoer"]>=0.075, "nieuwe_categorie"] = 1

In [ ]:
edges["source"].unique()

In [ ]:
edges_hydroobjecten = edges[edges["source"] == "hydroobject"][["nieuwe_categorie", "geometry"]]
edges_overige_watergang = edges[edges["source"] == "overige_watergang"][["nieuwe_categorie", "geometry"]]

In [ ]:
edges_overige_watergang

In [ ]:
overig = res.overige_watergang.sjoin_nearest(edges_overige_watergang, how="left", max_distance=1.0)
overig["lengte"] = overig.geometry.length
res_overig = overig.groupby("nieuwe_categorie")["lengte"].sum()
res_overig

In [ ]:
res_overig.values

In [ ]:
overig["nieuwe_categorie"].unique()

In [ ]:
overig.lengte.sum()

In [ ]:
overig.lengte.sum() - sum(res_overig.values)

In [ ]:
hydro = res.hydroobject.sjoin_nearest(edges_hydroobjecten, how="left", max_distance=1.0)
hydro["lengte"] = hydro.geometry.length
res_hydro1 = hydro[hydro["categorieoppwaterlichaam"]==1].groupby("nieuwe_categorie")["lengte"].sum()
res_hydro2 = hydro[hydro["categorieoppwaterlichaam"]==2].groupby("nieuwe_categorie")["lengte"].sum()
res_hydro1

In [ ]:
res_hydro2

In [ ]:
res_hydro1.values

In [ ]:
res_hydro2.values

In [ ]:
hydro["nieuwe_categorie"].unique()

In [ ]:
hydro[hydro["categorieoppwaterlichaam"]==1].lengte.sum()

In [ ]:
hydro[hydro["categorieoppwaterlichaam"]==2].lengte.sum()